In [4]:
import collections
import pathlib
import matplotlib
import tensorflow as tf
import random
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import tensorflow_datasets as tfds
import tensorflow_text as tf_text

In [5]:
matplotlib.rcParams['figure.figsize'] = [12, 8]
np.set_printoptions(precision=3, suppress=True)

seed = 22
tf.random.set_seed(seed)
hidden_neurons = 16
number_of_hidden_layers = 2
batch_size = 32
learning_rate = 0.001
max_epochs = 100
k = 10
train_val_split = 0.6
activation_func = 'relu'
max_words = 10000

In [232]:
descriptions = pd.read_csv('descriptions.csv')
genres = pd.read_csv('genres.csv')

In [233]:
_ = genres.pop('Unnamed: 0')
_ = descriptions.pop('Unnamed: 0')

In [234]:
df = pd.concat([descriptions, genres], axis=1)

In [235]:
column_indices = {name: i for i, name in enumerate(df.columns)}

n = len(df)
train_df = df[0:int(n*0.4)]
val_df = df[int(n*0.4):int(n*0.7)]
test_df = df[int(n*0.7):]

In [236]:
train_df = train_df.sample(frac=1, random_state=seed).reset_index(drop=True)
val_df = val_df.sample(frac=1, random_state=seed).reset_index(drop=True)
test_df = test_df.sample(frac=1, random_state=seed).reset_index(drop=True)

In [237]:
_ = test_df.pop('genres')

In [148]:
number_of_genres = 0
list_of_genres = []
for i in range(len(genres)):
    string = ''.join(genres.iloc[i])
    new_string = string.replace('[', '')
    new_string = new_string.replace(']', '')
    new_string = new_string.replace("'", '')
    new_string = new_string.replace("'", '')
    new_string = new_string.split(", ")
    for j in range(len(new_string)):
        if new_string[j] not in list_of_genres:
            list_of_genres.append(new_string[j])
            number_of_genres += 1